# Generador de personas aleatorias

In [5]:
import math
import random as rm
import numpy as np
import pandas as pd
import datetime as dt
import pytz
from faker import Faker
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from time import sleep
from funciones import quitar_acentos, sacar_curp, sacar_signo, complexion_fisica

# Instancia para usar servicio de geoubicaciones
geolocator = Nominatim(user_agent = 'geoapiExercises')
geocode = lambda query: geolocator.geocode(query, featuretype = 'city', language = 'es')
reverse = lambda coords: geolocator.reverse(coords, language = 'es')

# Obtener coordenadas del centro de una ciudad
def obtener_coordenadas(ciudad, estado, pais = 'Mexico') -> dict:
	print(f'\033[36mBuscando coordenadas de {ciudad}, {estado}, {pais}\033[0m')
	try:
		# Hacer peticion a un api para obtener coordenadas del centro y extremas
		location = geocode(f'{ciudad} {estado} {pais}')
		# En caso de que no se haya encontrado nada
		if not location: raise BaseException(f'No se encontraron coordenadas')
		location = location.raw
		# Obtener coordenadas del centro
		centro = (float(location['lat']), float(location['lon']))
		# Obtener extremos originales
		minx, maxx, miny, maxy = [float(i) for i in location['boundingbox']]
		ne, no, se, so = [(maxx, maxy), (maxx, miny), (minx, maxy), (minx, miny)]
		print(f'\033[33mobtener_coordenadas()\033[0m: Coordenadas encontradas')
	except BaseException as e:
		print(f'\033[31mobtener_coordenadas()\033[0m:', e)
		return {}
	return {'centro': centro, 'noreste': ne, 'noroeste': no, 'sureste': se, 'suroeste': so}

# Generar n puntos random alrededor de un area
def generar_puntos_random(coords: dict, n = 1, reducir = True) -> list:
	# Ver si estan los 3 puntos necesarios
	coords_necesarias = ('centro', 'suroeste', 'noreste')
	if not all(k in coords for k in coords_necesarias):
		print('\033[33mgenerar_puntos_random()\033[0m: No se encontro alguna de las coordenadas necesarias:', coords_necesarias)
		return []
	# Obtener coordenadas del centro
	cenx, ceny = coords['centro']
	# Obtener extremos originales
	minx, miny = coords['suroeste']
	maxx, maxy = coords['noreste']
	# Hallar punto medio entre (suroeste centro) y (centro noreste)
	minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
	maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# True para reducir el area de la ciudad
	if reducir:
		# Hallar punto medio de puntos medios y centro
		minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
		maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# Obtener n puntos aleatorios dentro del area
	puntos_random = []
	while len(puntos_random) < n:
		puntos_random.append((rm.uniform(minx, maxx), rm.uniform(miny, maxy)))
	return puntos_random

# Buscar una direccion a partir de una lista de corrdenadas
def buscar_direccion(coords: dict) -> dict:
	# Generar puntos aleatorios alrededor del area
	puntos_random = generar_puntos_random(coords, 5)
	if not puntos_random: return {}
	dirrr = {}
	listo = False
	print('\033[36mBuscando direccion\033[0m')
	for punto_random in puntos_random:
		dirr = {}
		try:
			address = reverse(punto_random)
			if not address: continue
			direccion = address.raw['address']
			print(direccion)
			# Ver que campos se encontraron
			# https://nominatim.org/release-docs/latest/api/Output/#addressdetails
			# local
			if any(k in direccion for k in ('amenity', 'building', 'shop', 'office', 'leisure')):
				if 'amenity' in direccion: dirr['local'] = direccion['amenity']
				elif 'building' in direccion: dirr['local'] = direccion['building']
				elif 'shop' in direccion: dirr['local'] = direccion['shop']
				elif 'office' in direccion: dirr['local'] = direccion['office']
				elif 'leisure' in direccion: dirr['local'] = direccion['leisure']
			# calle
			if 'road' in direccion: dirr['calle'] = direccion['road']
			# numero
			if 'house_number' in direccion: dirr['numero'] = direccion['house_number']
			else: dirr['numero'] = 'S/N'
			# colonia
			if any(k in direccion for k in ('neighbourhood', 'residential', 'suburb', 'quarter')):
				if 'neighbourhood' in direccion: dirr['colonia'] = direccion['neighbourhood']
				elif 'residential' in direccion: dirr['colonia'] = direccion['residential']
				elif 'suburb' in direccion: dirr['colonia'] = direccion['suburb']
				elif 'quarter' in direccion: dirr['colonia'] = direccion['quarter']
			# codigo postal
			if 'postcode' in direccion: dirr['cp'] = direccion['postcode']
			# localidad
			if any(k in direccion for k in ('city', 'town', 'village', 'hamlet')):
				if 'city' in direccion: dirr['localidad'] = direccion['city']
				elif 'town' in direccion: dirr['localidad'] = direccion['town']
				elif 'village' in direccion: dirr['localidad'] = direccion['village']
				elif 'hamlet' in direccion: dirr['localidad'] = direccion['hamlet']
				elif 'isolated_dwelling' in direccion: dirr['localidad'] = direccion['isolated_dwelling']
			# municipio
			if 'county' in direccion: dirr['municipio'] = direccion['county']
			# estado
			if 'state' in direccion: dirr['estado'] = direccion['state']
			# pais
			if 'country' in direccion: dirr['pais'] = direccion['country']
			# Si se encontro al menos calle y cp
			if all(k in dirr for k in ('calle', 'numero')) or 'cp' in dirr: listo = True
			# Si ya se encontro una buena direccion, dejar de buscar
			if listo:
				print('\033[32mbuscar_direccion()\033[0m: Direccion encontrada en', punto_random)
				print('\033[32m', dirr, '\033[m')
				dirrr = dirr
				break
		except BaseException as e:
			print(f'\033[31mbuscar_direccion()\033[0m:', e)
			return {}
	if not listo: print('\033[33mbuscar_direccion()\033[0m: No se encontro una direccion precisa')
	return dirrr

# Calcular cuadro delimitador a partir de una cordenada dada y su amplitud
def bounding_box(coords: tuple, amplitud_en_km = 6) -> list:
	# grados a radianes
	def deg2rad(degrees): return math.pi * degrees / 180.0

	# radianes a grados
	def rad2deg(radians): return 180.0 * radians / math.pi

	# Calcular radio de la tierra (R) en metros con una latitud dada segun WGS-84
	def radio_terrestre_wgs84(lat: float) -> float:
		# http://en.wikipedia.org/wiki/Earth_radius
		# Semiejes de WGS-84
		semieje_mayor_a = 6378137.0  # Semieje mayor a [metros]
		semieje_menor_b = 6356752.31424  # Semieje menor b [metros]
		# Sacar R
		an = (semieje_mayor_a ** 2) * math.cos(lat)
		bn = (semieje_menor_b ** 2) * math.sin(lat)
		ad = semieje_mayor_a * math.cos(lat)
		bd = semieje_menor_b * math.sin(lat)
		return math.sqrt((an ** 2 + bn ** 2) / (ad ** 2 + bd ** 2))

	# convertir latitud y longitud a radianes
	lat_rad = deg2rad(coords[0])
	lon_rad = deg2rad(coords[1])
	# ampliar el area de busqueda
	amplitud = 1000 * amplitud_en_km

	# Radio terrestre en la latitud dada
	radius = radio_terrestre_wgs84(lat_rad)
	# Radio paralelo en la latitud
	pradius = radius * math.cos(lat_rad)

	# Calcular limites y convertirlos a grados
	lat_min = rad2deg(lat_rad - amplitud / radius)
	lon_min = rad2deg(lon_rad - amplitud / pradius)
	lat_max = rad2deg(lat_rad + amplitud / radius)
	lon_max = rad2deg(lon_rad + amplitud / pradius)
	return [(lat_min, lon_min), (lat_max, lon_max)]

In [4]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
pge = pd.read_csv('./conjunto_datos/porcentajes_grasa_edad.csv')
ct_complx = pd.read_csv('./catalogos/edad_pg_complexion.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv', header = None)
file_ciudades = './catalogos/ciudades.csv'
ct_ciudades = pd.read_csv(file_ciudades)

def personas_aleatorias(n = 5):
	fake = Faker(['es_MX'])
	personas = []
	for i in range(n):
		print(f'\033[34mIdx{i}\033[0m')
		sexo = rm.choice(['H', 'M'])
		# generar tipo de sangre y asignar probabilidad a cada valor para que salga
		tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], [30, 6, 9, 2, 4, 1, 39, 9])[0]

		fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()), maximum_age = 90)
		# calcular edad en base a la fecha de nacimiento
		dia_actual = dt.date.today()
		edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

		# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
		# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
		# mucha gente deja de crecer a los 20
		if edad <= reap['edad'].max(): criter = (reap['edad'] == edad)
		else: criter = (reap['edad'] == reap['edad'].max())
		margen_ap = reap[criter].to_dict(orient = 'records')[0]

		# Dependiendo de la edad se busca el rango de % de grasa
		margen_pge = pge[(pge['li.edad'] <= edad) & (pge['ls.edad'] >= edad)].to_dict(orient = 'records')[0]

		# Se filtra por edad los criterios para saber la complexion fisica
		# mas adelante se checara el sexo y en que intervalo esta su % de grasa
		comp_criter = (ct_complx['li.edad'] <= edad) & (ct_complx['ls.edad'] >= edad)

		# dependiendo del sexo se generan nombres femeninos o masculinos
		# y estaturas y pesos dentro de su margen correspondiente
		if sexo == 'H':
			titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['h'])
			altura = round(rm.uniform(margen_ap['li.altura.h.cm'], margen_ap['ls.altura.h.cm']), 2)
			peso = round(rm.uniform(margen_ap['li.peso.h.kg'], margen_ap['ls.peso.h.kg']), 2)
			grasa = round(rm.uniform(margen_pge['li.pg.h'], margen_pge['ls.pg.h']), 1)
			comp_criter = comp_criter & (ct_complx['li.pg.h'] <= grasa) & (ct_complx['ls.pg.h'] >= grasa)
		else:
			titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['m'])
			altura = round(rm.uniform(margen_ap['li.altura.m.cm'], margen_ap['ls.altura.m.cm']), 2)
			peso = round(rm.uniform(margen_ap['li.peso.m.kg'], margen_ap['ls.peso.m.kg']), 2)
			grasa = round(rm.uniform(margen_pge['li.pg.m'], margen_pge['ls.pg.m']), 1)
			comp_criter = comp_criter & (ct_complx['li.pg.m'] <= grasa) & (ct_complx['ls.pg.m'] >= grasa)

		complexion = ct_complx[comp_criter]['complexion'].values[0]
		mins_ejerc_dia = round(rm.normalvariate(60, 70))
		if mins_ejerc_dia < 0: mins_ejerc_dia = 0

		apellido_p = fake.last_name()
		apellido_m = fake.last_name()

		id_estado = rm.choice(ct_ciudades['id_estado'].unique())
		ct_ciudad = ct_ciudades[ct_ciudades['id_estado'] == id_estado].sample()
		ct_ciudad_i_row = ct_ciudad.index.values[0]
		ct_ciudad = ct_ciudad.to_dict(orient = 'records')[0]

		ciudad = ct_ciudad['ciudad']
		estado = ct_ciudad['estado']
		tel = '({:d}) {:03d} {:04d}'.format(ct_ciudad['lada'], rm.randint(0, 999), rm.randint(0, 9999))
		curp = sacar_curp(nombre, apellido_p, apellido_m, sexo, fecha_nac, estado)
		signo = sacar_signo(fecha_nac)
		religion = rm.choices(['Catolico', 'Cristiano', 'Otros', 'Ateo'], [78, 11, 3, 8])[0]
		profesion = profesiones.sample().values[0][0] if edad >= 18 else None

		"""Correo"""
		# Generar un nombre de usuario para un email
		switcher = {
			1: fake.user_name(),
			# aperez
			2: f'{nombre[0]}{apellido_p}',
			# alonso_perez
			3: f'{nombre}_{apellido_p}',
			# aps70
			4: f'{nombre[0]}{apellido_p[0]}{apellido_m[0]}{fecha_nac.strftime("%y")}',
			# aps1970
			5: f'{nombre[0]}{apellido_p[0]}{apellido_m[0]}{fecha_nac.year}',
			# alonsoperezsoltero
			6: f'{nombre}{apellido_p}{apellido_m}'
		}
		rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
		usuario = switcher.get(rand_int, 'usuario')
		# quitar espacios o reemplazarlos por un _
		usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
		usuario = quitar_acentos(usuario)
		subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
		correo = usuario + f'@example{subdominio}'

		"""
		El Número de Seguridad Social (NSS) es una homoclave que asigna el IMSS
		para llevar un registro de los trabajadores y asegurados
		que están inscritos ahi mismo. El NSS es único, permanente e intransferible.
		"""
		# El NSS está compuesto por 11 dígitos:
		nss = None
		if profesion:
			# TODO Los primeros dos caracteres están vinculados a la subdelegación en el que la persona fue afiliada.
			nss1 = ('%02d' % rm.randint(0, 99))
			# Los dos dígitos siguientes indican el año en el que la persona se afilió al Seguro Social.
			# suponiendo que la persona empezo a trabajar de sus 18 a sus 25
			nss2 = (fecha_nac + pd.DateOffset(months = rm.randint(216, 300))).strftime('%y')
			# Los siguientes dos dígitos corresponden a la fecha de nacimiento del afiliado.
			nss3 = fecha_nac.strftime('%y')
			# Los cuatro números siguientes son los dígitos que asigna el IMSS al trabajador.
			nss4 = ('%04d' % rm.randint(0, 9999))
			# El último dígito corresponde al número de verificación del trabajador en el IMSS.
			nss5 = str(rm.randint(0, 9))
			nss = f'{nss1} {nss2} {nss3} {nss4} {nss5}'

		"""Coordenadas de la ciudad"""
		# si el catalogo ya tiene coordenadas, tomar esos valores
		if pd.notna(ct_ciudad['centro']) and pd.notna(ct_ciudad['suroeste']) and pd.notna(ct_ciudad['noreste']):
			coords = {
				'centro': eval(ct_ciudad['centro']),
				'suroeste': eval(ct_ciudad['suroeste']),
				'noreste': eval(ct_ciudad['noreste'])
			}
		else:
			# si no, buscar en internet el punto central y extremos segun la ciudad
			coords = obtener_coordenadas(ciudad, estado)
			if coords:
				# guardar coordenadas en el catalogo
				# para que a la otra no haya que buscar en internet
				ct_ciudades.loc[ct_ciudad_i_row, 'centro'] = str(coords['centro'])
				ct_ciudades.loc[ct_ciudad_i_row, 'suroeste'] = str(coords['suroeste'])
				ct_ciudades.loc[ct_ciudad_i_row, 'noreste'] = str(coords['noreste'])
				ct_ciudades.to_csv(file_ciudades, index = False)

		"""Direccion"""
		direcc = cp = None
		if 'centro' in coords:
			direccion = buscar_direccion(coords)
			if direccion:
				cp = direccion['cp'] if 'cp' in direccion else None
				if all(k in direccion for k in ('calle', 'numero')):
					direcc = f"{direccion['calle']}, {direccion['numero']}"
					if 'colonia' in direccion: direcc += f", {direccion['colonia']}"

		"""Zona horaria"""
		timezone = tz_offset = None
		if 'centro' in coords:
			# obtener zona horaria segun latitud y longitud
			obj = TimezoneFinder()
			#if centro[1] > 180.0 or centro[1] < -180.0 or centro[0] > 90.0 or centro[0] < -90.0:
			timezone = obj.timezone_at(lat = coords['centro'][0], lng = coords['centro'][1])
			# en Enero no hay DST
			#tz_offset = pytz.timezone(timezone).localize(dt.datetime(2011, 1, 1)).strftime('%z')
			# 'datetime.now' tiene DST (Daylight Saving Time)
			tz_offset = dt.datetime.now(pytz.timezone(timezone)).strftime('%z')

		if not 'centro' in coords: coords['centro'] = (np.nan, np.nan)

		personas.append({
			'titulo': titulo,
			'nombre': nombre,
			'apellido_p': apellido_p,
			'apellido_m': apellido_m,
			'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
			'tipo_sangre': tipo_sangre,
			'sexo': sexo,
			'altura': altura,
			'peso': peso,
			'complexion': complexion,
			'porcent_grasa': grasa,
			'mins_ejerc_dia': mins_ejerc_dia,
			'color_fav': fake.safe_color_name(),
			'signo': signo,
			'religion': religion,
			'curp': curp,
			'correo': correo,
			'tel': tel,
			'profesion': profesion,
			'nss': nss,
			'direccion': direcc,
			'cp': cp,
			'ciudad': ciudad,
			'estado': estado,
			'lat': coords['centro'][0],
			'lon': coords['centro'][1],
			'zona_horaria': timezone,
			'tz_offset': tz_offset,
		})

	return pd.DataFrame(personas)

personas_aleatorias(10)

Idx0
Buscando direccion
{'village': 'Pueblo Juárez (La Magdalena)', 'county': 'Coquimatlán', 'state': 'Colima', 'ISO3166-2-lvl4': 'MX-COL', 'postcode': '28410', 'country': 'México', 'country_code': 'mx'}
buscar_direccion(): Direccion encontrada en (19.186813721302677, -103.96458087881621)
 {'numero': 'S/N', 'cp': '28410', 'localidad': 'Pueblo Juárez (La Magdalena)', 'municipio': 'Coquimatlán', 'estado': 'Colima', 'pais': 'México'} 
Idx1
Buscando direccion
{'road': 'Paseo Álvaro Obregón', 'town': 'Todos Santos', 'county': 'Municipio de La Paz', 'state': 'Baja California Sur', 'ISO3166-2-lvl4': 'MX-BCS', 'postcode': '23300', 'country': 'México', 'country_code': 'mx'}
buscar_direccion(): Direccion encontrada en (23.4465791474966, -110.21569094252924)
 {'calle': 'Paseo Álvaro Obregón', 'numero': 'S/N', 'cp': '23300', 'localidad': 'Todos Santos', 'municipio': 'Municipio de La Paz', 'estado': 'Baja California Sur', 'pais': 'México'} 
Idx2
Buscando direccion
{'road': 'Avenida Benito Juárez', 

,titulo,nombre,apellido_p,apellido_m,fecha_nac,tipo_sangre,sexo,altura,peso,complexion,...,profesion,nss,direccion,cp,ciudad,estado,lat,lon,zona_horaria,tz_offset
0,None,Indira,Granados,Cardenas,1961-06-07,A+,M,171.42,78.16,Bueno,...,Ejecutivo de cuentas de publicidad,40 80 61 6755 8,None,28410,Coquimatlán,Colima,19.175978,-103.923913,America/Mexico_City,-0500
1,Ing.,Abigail,Rivero,Caballero,1962-01-11,O+,M,156.37,59.92,Bueno,...,Activista,50 80 62 7701 2,"Paseo Álvaro Obregón, S/N",23300,Todos Santos,Baja California Sur,23.450067,-110.225374,America/Mazatlan,-0600
2,Mtro.,Joaquín,Cotto,Garay,1972-10-17,AB+,H,180.79,79.39,Obeso,...,Periodista,46 96 72 0573 9,"Avenida Benito Juárez, S/N",73800,Ixtahuiata (La Legua),Puebla,19.843889,-97.335556,America/Mexico_City,-0500
3,None,Vanesa,Soliz,Razo,2009-12-18,O+,M,148.15,45.26,Media,...,None,None,"Carretera División del Norte-Candelaria, S/N",None,Candelaria,Campeche,18.186072,-91.043117,America/Merida,-0500
4,None,Enrique,Alarcón,Nava,1989-11-08,O+,H,170.06,85.72,Sobrepeso,...,Director de Estrategia,32 14 89 1331 0,"Calle Isabel la Católica, S/N",70110,Ixtepec,Oaxaca,16.556414,-95.095965,America/Mexico_City,-0500
5,None,Yandel,de León,Noriega,1957-05-09,A+,H,172.67,70.29,Media,...,Arqueólogo,55 79 57 5862 0,"Calle Pedro Abencio, S/N",None,Vizarrón de Montes,Querétaro,20.832502,-99.719802,America/Mexico_City,-0500
6,None,Nancy,Nazario,Negrete,1943-07-29,B+,M,158.17,55.85,Media,...,Acupunturista,71 65 43 6243 2,None,None,Punta Banda,Baja California,NaN,NaN,None,None
7,Sr.,Orlando,Anaya,Acosta,1957-05-27,O+,H,171.69,66.00,Media,...,Conservador de muebleria,65 76 57 0650 7,"Calle Ignacio Aldama, S/N",None,San Fernando,Tamaulipas,24.846677,-98.147349,America/Monterrey,-0500
8,Ing.,Bernabé,Manzanares,Tejeda,1952-08-10,A-,M,153.35,79.26,Sobrepeso,...,Gerente de servicio al cliente,41 73 52 8462 1,None,None,Marcelino Rodríguez,Morelos,NaN,NaN,None,None
9,Dra.,Fabiola,Mayorga,Villagómez,1984-11-24,O-,M,164.26,55.75,Media,...,Curador,87 08 84 9033 8,"Calle Primavera, S/N",74128,Santa María Moyotzingo,Puebla,19.245278,-98.404722,America/Mexico_City,-0500


In [ ]:
# Buscar coordenadas de cada ciudad y crear un csv con las ciudades que no se le pudieron encontrar una
def buscar_ciudades_perdidas():
	ciudades_perdidas = []
	for i, ciudad in ct_ciudades.iterrows():
		# si esta ciudad no tiene estos tres campos
		no_hay_centro = pd.isna(ciudad['centro'])
		no_hay_suroeste = pd.isna(ciudad['suroeste'])
		no_hay_noreste = pd.isna(ciudad['noreste'])
		if (no_hay_centro and no_hay_suroeste and no_hay_noreste) or (no_hay_suroeste and no_hay_noreste):
			print(f'\033[34mIdx{i}\033[0m')
			# para no hacer peticiones muy seguido
			sleep(5)
			# buscar en internet el punto central y extremos segun la ciudad
			coordenadas = obtener_coordenadas(ciudad['ciudad'], ciudad['estado'])
			if coordenadas:
				centro = coordenadas['centro']
				# guardar coordenadas en el catalogo
				# para que a la otra no haya que buscar en internet
				ct_ciudades.loc[i, 'centro'] = str(centro)
				ct_ciudades.loc[i, 'suroeste'] = str(coordenadas['suroeste'])
				ct_ciudades.loc[i, 'noreste'] = str(coordenadas['noreste'])
				ct_ciudades.to_csv(file_ciudades, index = False)
			else:
				ciudades_perdidas.append({
					'idx': i,
					'estado': ciudad['estado'],
					'cuidad': ciudad['ciudad']
				})
	df = pd.DataFrame(ciudades_perdidas)
	df.to_csv('./catalogos/ciudades_perdidas.csv', index = False)
	print('\033[34mListo\033[0m')